# Installing & Imports

In [1]:
conda install pandas==0.24.2

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - pandas==0.24.2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pandas-0.24.2              |   py36he6710b0_0         8.5 MB
    ------------------------------------------------------------
                                           Total:         8.5 MB

The following packages will be DOWNGRADED:

  pandas                               1.0.1-py36h0573a6f_0 --> 0.24.2-py36he6710b0_0



pandas-0.24.2        | 8.5 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [2]:
conda upgrade --all -y

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bottleneck-1.3.2           |   py36heb32a55_0         124 KB
    dask-2.11.0                |             py_0          13 KB
    dask-core-2.11.0           |             py_0         565 KB
    distributed-2.11.0         |           py36_0         943 KB
    gunicorn-20.0.4            |           py36_0         125 KB
    hypothesis-5.5.4           |             py_0         227 KB
    nb_conda_kernels-2.2.2     |           py36_0          39 KB
    ncurses-6.2                |       he6710b0_0         1.1 MB
    plotly-4.5.2               |             py_0         1.6 MB
    pytest-astropy-0.8.0       |             py_0           9 KB
    ------------------------------------------------------------
                                         

In [3]:
conda install snorkel==0.9.0 -c conda-forge

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - snorkel==0.9.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    cudatoolkit-10.0.130       |                0       261.2 MB
    cudnn-7.6.5                |       cuda10.0_0       165.0 MB
    libprotobuf-3.11.3         |       h8b12597_0         4.8 MB  conda-forge
    ninja-1.10.0               |       hc9558a2_0         1.9 MB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    pandas-0.24.2              |   py36hb3f55d8_1        11.1 MB  conda-forge
    protobuf-3.11.3       

In [4]:
conda install -c conda-forge textblob

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - textblob


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    textblob-0.15.3            |             py_0         595 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         595 KB

The following NEW packages will be INSTALLED:

  textblob           conda-forge/noarch::textblob-0.15.3-py_0



textblob-0.15.3      | 595 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [5]:
from snorkel.labeling import LFAnalysis
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier,LabelModel
from snorkel.preprocess import preprocessor
from textblob import TextBlob
import pandas as pd

# Reading & Preparing TSV file 

In [6]:
# Read TSV file
tsv_reviews = pd.read_csv('/project/reviews.tsv', sep='\t', header=0, encoding='unicode_escape')

In [7]:
tsv_reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [8]:
# Extract review and fresh columns
tsv_reviews = pd.DataFrame(tsv_reviews, columns = ['review', 'fresh'])

In [9]:
tsv_reviews.head()

,review,fresh
0,A distinctly gallows take on contemporary fina...,fresh
1,It's an allegory in search of a meaning that n...,rotten
2,... life lived in a bubble in financial dealin...,fresh
3,Continuing along a line introduced in last yea...,fresh
4,... a perverse twist on neorealism...,fresh


In [10]:
tsv_reviews.isnull().sum()

review    5563
fresh        0
dtype: int64

In [11]:
# drop NaN rows in reviews
index_name = tsv_reviews[(tsv_reviews['review'].isnull())].index
tsv_reviews.drop(index_name, inplace= True)

In [12]:
tsv_reviews.isnull().sum()

review    0
fresh     0
dtype: int64

In [13]:
# rename fresh as 1 and rotten as 0
tsv_reviews['fresh'].replace({'fresh':'1', 'rotten':'0'}, inplace = True)

In [39]:
#Rename columns
tsv_reviews.rename(columns={'fresh':'Freshness','review':'Review'},inplace=True)
tsv_reviews = tsv_reviews.sample(5000)
#take 5000
tsv_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 785 to 53651
Data columns (total 2 columns):
Review       5000 non-null object
Freshness    5000 non-null object
dtypes: object(2)
memory usage: 117.2+ KB


# Reading & Preparing CSV file 

In [15]:
# Read CSV file
csv_reviews= pd.read_csv('/project/rotten_tomatoes_reviews.csv')
csv_reviews.head()


,Freshness,Review
0,1,"Manakamana doesn't answer any questions, yet ..."
1,1,Wilfully offensive and powered by a chest-thu...
2,0,It would be difficult to imagine material mor...
3,0,Despite the gusto its star brings to the role...
4,0,If there was a good idea at the core of this ...


In [40]:
#Swap Freshness and Review 
columns_titles = ["Review","Freshness"]
csv_reviews=csv_reviews.reindex(columns=columns_titles)
csv_reviews = csv_reviews.sample(5000)

csv_reviews.head()
csv_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 405590 to 328471
Data columns (total 2 columns):
Review       5000 non-null object
Freshness    5000 non-null int64
dtypes: int64(1), object(1)
memory usage: 117.2+ KB


# Web Scraping & Preparing scrapped data 

In [66]:
#Read Web Scraping Data
web_scraping_reviews= pd.read_csv('/project/web_scraping_rotten_tomatoes_part2.csv')
web_scraping_reviews.head()

web_scraping_reviews = web_scraping_reviews.sample(5000)

In [67]:
# rename fresh as 1 and rotten as 0
web_scraping_reviews['rating'].replace({'fresh':'1', 'rotten':'0'}, inplace = True)

#Rename Rating to Review 
web_scraping_reviews.rename(columns={'rating':'Freshness', 'review':'Review'},inplace=True)
web_scraping_reviews.head()

# Extract Review and Freshness columns
web_scraping_reviews= pd.DataFrame(web_scraping_reviews, columns = ['Review', 'Freshness'])

In [68]:
web_scraping_reviews.head()

,Review,Freshness
424,"Mine 9 juggles too many characters, but it wor...",1
1294,With its avid belief in the fundamental goodne...,1
1239,These two ferociously talented women are just ...,1
688,Where's My Roy Cohn? could be a soap opera if ...,1
5498,Roxanne Benjamin's protagonist Wendy has to fa...,0


# Combining all the data together 

In [69]:
# csv_reviews.info()
# tsv_reviews.info()
# web_scraping_reviews.info()



# Concat two files into all_reviews
all_reviews=pd.concat([csv_reviews, tsv_reviews,web_scraping_reviews],axis=0, sort=False)
all_reviews.head()

,Review,Freshness
405590,Deceptively deep in both its sumptuous visual...,1
282529,"The story may sound intolerably precious, but...",1
299416,"Class, competition, and the world's dumbest c...",1
83443,Argentinean road trip movie takes a very slow...,0
56790,"""The Dressmaker"" is only sew-sew.",0


In [70]:
all_reviews.shape

(15000, 2)

# Split into test and training set 

In [82]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(all_reviews, test_size=0.2)

In [83]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 18160 to 54456
Data columns (total 2 columns):
Review       12000 non-null object
Freshness    12000 non-null object
dtypes: object(2)
memory usage: 281.2+ KB


In [84]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 3678 to 27216
Data columns (total 2 columns):
Review       3000 non-null object
Freshness    3000 non-null object
dtypes: object(2)
memory usage: 70.3+ KB


In [85]:
#For training set - need to remove the labels (so we have an unlabelled training set)
train = train.drop('Freshness', 1)

In [86]:
train.head()

,Review
18160,Devastating ... Heaven only knows why Deliver ...
5206,After a strong emotional start which is well a...
3668,Shamelessly self-indulgent film-as-therapy for...
265809,A brilliant tour de force of trash cinema.
86,"Things become really strange, violent and ridi..."


In [87]:
#From labelled test set, extract a sample to find out about which labelling functions could be written
#Not sure how big the development split_ can be --> take sample of 1000 data points 

development_split = test.sample(1000,random_state=42)
development_split.head()


,Review,Freshness
258266,It took a lot of bravery and candor to put th...,1
27084,For all of the massive sets and overstated rum...,0
238856,Their Reservoir Dogs-a multi-character story ...,1
1482,"The tension builds, the special effects are gr...",1
190724,Pleasantville seemed to say that such squeaky...,1


In [88]:
#For finding labelling functions: 
development_split

,Review,Freshness
258266,It took a lot of bravery and candor to put th...,1
27084,For all of the massive sets and overstated rum...,0
238856,Their Reservoir Dogs-a multi-character story ...,1
1482,"The tension builds, the special effects are gr...",1
190724,Pleasantville seemed to say that such squeaky...,1
116135,"The Most Hated Woman isn't a documentary, and...",1
5396,"JT LeRoy is a decent telling of a fascinating,...",1
3711,It's a strange and enticing mix: a superhero o...,1
35733,A second look reveals everything wrong with th...,1
116671,What happens when a seemingly righteous opera...,1


In [89]:
development_split.to_csv('development_split.csv')

In [90]:
development_split.head()

,Review,Freshness
258266,It took a lot of bravery and candor to put th...,1
27084,For all of the massive sets and overstated rum...,0
238856,Their Reservoir Dogs-a multi-character story ...,1
1482,"The tension builds, the special effects are gr...",1
190724,Pleasantville seemed to say that such squeaky...,1


In [65]:
#Might have to get rid off index?